In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2


In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

# create an alias to the api
api = connect_widget.get_api()

Let's start with a simple portfolio and some unit allocations.

In [ ]:
portfolio = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}

Next, fetch the asset master and map our portfolio to the Serenity internal IDs.

In [ ]:
refdata_response = client.call_api(
    api_group='refdata',
    api_path='/asset/summaries'
)

df = pd.json_normalize(refdata_response['assetSummary']).set_index('assetId')
asset_master_df = df.filter(['assetId', 'nativeSymbol'])
ids_dict = {asset_master_df[asset_master_df['nativeSymbol'] == s].index[0]: s for s in portfolio.keys()}
portfolio_ids = {asset_master_df[asset_master_df.nativeSymbol == id].index[0]:qty for (id, qty) in portfolio.items()}
portfolio_ids

We'll use these Serenity asset IDs to build the query for the Serenity client.

In [ ]:
portfolio_input = {}
portfolio_input['assetPositions'] = [{'assetId': item[0], 'quantity': item[1]} for item in portfolio_ids.items()]

Note that in this exercise we do a point in time analysis, which is how the [Serenity API](https://developer.athansor.cloudwall.network/api/serenity-risk-services) is supposed to be used. If you need a timeseries of these data, you will need to loop through a set of dates.

In [ ]:
from serenity_sdk.client import CallType

params = {'as_of_date': '2021-07-01'}

risk_attribution_json = client.call_api(
    api_group='risk',
    api_path='/compute/attribution',
    body_json=portfolio_input,
    params=params,
    call_type=CallType.POST
)

The API response will be a JSON file, with the information of the request, reference date, and the risk attribution.

In [ ]:
risk_attribution_json.keys()

Initially, we want to see how much of the portfolio volatility is explained by the risk factors, and what is the specific (idiosyncratic) risk. This latter is the part of the portfolio volatility specific to the assets in the portfolio. This means they cannot be explained by the used risk factors. We usually want to see most of the risk explained by the risk factors, which means the sources of risk in our portfolio are known. We can do this decomposition for both volatility and for the variance.

In [ ]:
total_risk = pd.DataFrame.from_dict(risk_attribution_json['totalRisk'])
total_risk_table = total_risk.style.format("{:.1%}").bar()

total_risk_table

The next step is to get marginal risk, which represents the percentage contribution of each asset’s volatility to the portfolio volatility. We can use this to understand what is the marginal contribution to the risk of each asset. *Marginal* means that it’s the impact on the portfolio volatility of a % increase of a certain asset’s weight.

In [ ]:
marginal_risk = pd.DataFrame.from_dict(
    risk_attribution_json['assetMarginalRisk'])
marginal_risk.drop(columns=['sectorLevels'], inplace=True)
marginal_risk.set_index('assetId', inplace=True)
marginal_risk.set_index(marginal_risk.index.map(ids_dict), inplace=True)
marginal_risk_table = marginal_risk.style.format("{:.1%}").bar()

marginal_risk_table

We can weigh the marginal contribution by the asset weights to obtain the absolute contribution to the portfolio volatility of each asset. We can use this to understand how each asset is contributing to the volatility of the portfolio, and then we can make portfolio allocation decisions based on this information. 

In [ ]:
contribution_risk_absolute_byasset = pd.DataFrame.from_dict(
    risk_attribution_json['absoluteContributionRisk']['byAsset'])
contribution_risk_absolute_byasset.set_index('assetId', inplace=True)
contribution_risk_absolute_byasset.drop(columns=['sectorLevels'], inplace=True)
contribution_risk_absolute_byasset.set_index(contribution_risk_absolute_byasset.index.map(ids_dict), inplace=True)
contribution_risk_absolute_byasset_table = contribution_risk_absolute_byasset.style.format("{:.1%}").bar()

contribution_risk_absolute_byasset_table

In [ ]:
contribution_risk_absolute_byasset = pd.DataFrame.from_dict(
    risk_attribution_json['absoluteContributionRisk']['byAsset'])
contribution_risk_absolute_byasset.drop(columns=['sectorLevels'], inplace=True)
contribution_risk_absolute_byasset.set_index('assetId', inplace=True)
contribution_risk_absolute_byasset.set_index(contribution_risk_absolute_byasset.index.map(ids_dict), inplace=True)
contribution_risk_absolute_byasset_table = contribution_risk_absolute_byasset.style.format("{:.1%}").bar()

contribution_risk_absolute_byasset_table

Note how the absolute risk contribution sums up to the total volatility of the portfolio.

In [ ]:
contribution_risk_absolute_byasset.sum(axis=0)

Given the derivation of this metric, we can use it to back-off our portfolio weights.

In [ ]:
portfolio_weights = contribution_risk_absolute_byasset[['factorRisk']] / marginal_risk[['factorRisk']]
portfolio_weights.rename(columns={"factorRisk": 'weights'}, inplace=True)
portfolio_weights_table = portfolio_weights.style.format("{:.1%}").bar()

portfolio_weights_table

In a similar way we can decompose the portfolio *variance* to obtain the relative contribution to the portfolio risk. This helps to understand how much, in percentage terms, each asset is contributing to Factor, Specific and Total risk of the portfolio in terms of the variance. If compared to the absolute contribution, here we use variance because it’s additive, while volatility is not additive. This can help us to see if a certain asset has e.g. more Specific risk, which we might want to diversify or get exposure to. *Relative* here means that we are making a comparison to the other assets in the portfolio.

In [ ]:
contribution_risk_relative_byasset = pd.DataFrame.from_dict(
    risk_attribution_json['relativeContributionRisk']['byAsset'])
contribution_risk_relative_byasset.set_index('assetId', inplace=True)
contribution_risk_relative_byasset.drop(columns=['sectorLevels'], inplace=True)
contribution_risk_relative_byasset.set_index(contribution_risk_relative_byasset.index.map(ids_dict), inplace=True)
contribution_risk_relative_byasset_table = contribution_risk_relative_byasset.style.format("{:.1%}").bar()

contribution_risk_relative_byasset_table

And here we can also show that this metric sums up to the total portfolio risk (in terms of variance this time).

In [ ]:
contribution_risk_relative_byasset.sum(axis=0)

In [ ]:
factor_risk_test = total_risk.loc['factorRisk']['variance'] / total_risk.loc['totalRisk']['variance']
specific_risk_test = total_risk.loc['specificRisk']['variance'] / total_risk.loc['totalRisk']['variance']

print(factor_risk_test.round(10) == contribution_risk_relative_byasset.sum(axis=0)['factorRisk'].round(10))
print(specific_risk_test.round(10) == contribution_risk_relative_byasset.sum(axis=0)['specificRisk'].round(10))

Here's a way to get these metrics by sector. In this model we used the [DACS sector taxonomy](https://www.coindesk.com/static/coindesk-dacs/), with three sub-levels. The `parentSector` column tells us if the specific row has a parent sector. You can extract the full taxonomy from the `/sector/taxonomies` endpoint.

In [ ]:
contribution_risk_relative_bysector = pd.DataFrame.from_dict(
    risk_attribution_json['relativeContributionRisk']['bySector'])
    
contribution_risk_relative_bysector

Now we can also get the marginal, absolute, and relative contributions to the portfolio risk from each of the factors. The definitions of these metrics are the same as above. Here you can also get the factor exposure, which is the dot product between asset weights and each asset's factor loading. This, in practice, gives you the portfolio factor exposure in the selected point in time.

In [ ]:
factor_risk = pd.DataFrame.from_dict(risk_attribution_json['factorRisk'])
factor_risk = factor_risk.set_index('factor')
factor_risk.drop(columns=['factorExposure'], inplace=True)
factor_risk = factor_risk.transpose()
#factor_risk_table = factor_risk.style.format("{:.1%}").bar()

factor_risk

Note that the relative contribution sums to 100%, while the absolute contribution sums to the portfolio volatility explained by the factor risk.

In [ ]:
factor_risk.loc['relativeContribution'].sum().round(10) == 1

In [ ]:
factor_risk.loc['absoluteContribution'].sum().round(10) == total_risk['volatility']['factorRisk'].round(10)

For the derivations, we get the relative risk contribution from the absolute contribution. And we get the absolute contribution from the marginal contribution and from the factor exposure. If you want to know how to derive the marginal contribution, follow along, as it's explained later in this Notebook.

In [ ]:
factor_risk.loc['absoluteContribution'] / total_risk['volatility']['factorRisk'] == factor_risk.loc['relativeContribution']

From the Risk API you can also get the intermediate results that go into the factor model. Here you can, for instance, see the Factor-based asset covariance matrix and the residuals covariance matrix. These are fetched for the full universe, and we then subset it to the example portfolio.

In [ ]:
assetcov_response = client.call_api(
    api_group='risk',
    api_path='/asset/covariance',
    params=params
)

In [ ]:
cov_df = pd.DataFrame.from_dict(assetcov_response['matrix']).dropna()
cov_df = cov_df[cov_df['assetId1'].isin(ids_dict.keys()) & cov_df['assetId2'].isin(ids_dict.keys())]
cov_df = cov_df.pivot(index='assetId1', columns='assetId2', values='value')
cov_df.set_index(cov_df.index.map(ids_dict), inplace=True)
cov_df.columns = cov_df.columns.map(ids_dict)
sns.heatmap(cov_df, cmap='YlGnBu')

In [ ]:
rescov_response = client.call_api('risk', '/asset/residual/covariance', params=params)

In [ ]:
res_cov_df = pd.DataFrame.from_dict(rescov_response['matrix']).dropna()
res_cov_df = res_cov_df[res_cov_df['assetId1'].isin(ids_dict.keys()) & res_cov_df['assetId2'].isin(ids_dict.keys())]
res_cov_df = res_cov_df.pivot(index='assetId1', columns='assetId2', values='value')
res_cov_df.set_index(res_cov_df.index.map(ids_dict), inplace=True)
res_cov_df.columns = res_cov_df.columns.map(ids_dict)

res_cov_df

You can also fetch the factor returns that go into our risk factor model, in case you want to see how each factor performs at the selected point in time.

In [ ]:
factor_returns_response = client.call_api(
    api_group='risk',
    api_path='/factor/returns',
    params=params
)

In [ ]:
factor_returns = pd.DataFrame.from_dict(factor_returns_response['factorReturns']).pivot(index='closeDate', columns='factor', values='value')
factor_returns.style.format("{:.1%}")

We can also get the factor exposures of each asset in your portfolio.

In [ ]:
factor_exposures_response = client.call_api(
    api_group='risk',
    api_path='/asset/factor/exposures',
    params=params
)

In [ ]:
factor_exposures = pd.DataFrame.from_dict(factor_exposures_response['matrix'])
factor_exposures = factor_exposures[factor_exposures['assetId'].isin(ids_dict.keys())]
factor_exposures = factor_exposures.pivot(index='assetId', columns='factor', values='value')
factor_exposures.set_index(factor_exposures.index.map(ids_dict), inplace=True)

factor_exposures

Earlier we've mentioned that we can also derive our marginal risk factor contribution. For this, we need to fetch the factor covariance matrix, which is the covariance matrix derived from the factor returns.

In [ ]:
factorcov_response = client.call_api(
    api_group='risk',
    api_path='/factor/covariance',
    params=params
)

fac_cov_df = pd.DataFrame.from_dict(factorcov_response['matrix']).dropna()
fac_cov_df = fac_cov_df.pivot(index='factor1', columns='factor2', values='value')
sns.heatmap(fac_cov_df, cmap='YlGnBu')

fac_cov_df